<a href="https://colab.research.google.com/github/laraselinseyahi/Diabetic-Retinopathy-Classification-using-Deep-Learning/blob/main/CS230_Project_LaraSelinSeyahi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
import os


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# upload the datafiles to drive

%cd drive/MyDrive/retino
%ls

# Data Pre-processing


In [ ]:
train_dir = 'train/'
val_dir = 'valid/'
test_dir = 'test/'

# Image sizes
img_height = 224
img_width = 224
batch_size = 32

# Training dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

# Validation dataset
val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

# Test dataset
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False
)

In [ ]:
# helps keep memory in cache after it's been loaded from the disk
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
# image pre-processing
normalization_layer = tf.keras.applications.resnet.preprocess_input

In [ ]:
data_augmentation_layer = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

# ResNet-50


In [ ]:
# RESNET baseline model

# pre-trained model on imagenet
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in resnet_model.layers:
  layer.trainable = False

# on top of the base model, add layers for binary classification
inputs = tf.keras.Input(shape=(224, 224, 3))  # the input shape
x = data_augmentation_layer(inputs)  # data augmentation
x = normalization_layer(x)  # normalization (preprocessing)
x = resnet_model(x)  # passing through the base ResNet model
x = layers.GlobalAveragePooling2D()(x)  # average pooling layer
outputs = layers.Dense(1, activation='sigmoid')(x)  # sigmoid for binary classification

# create the model
model = models.Model(inputs, outputs)

base_learning_rate = 0.001

# since this is binary classification, loss is BCE
model.compile(optimizer=Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])

# train with frozen base model layers
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)

In [ ]:
# RESNET Experiment C

resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in resnet_model.layers:
  layer.trainable = False

inputs = tf.keras.Input(shape=(224, 224, 3))
x = data_augmentation_layer(inputs)
x = normalization_layer(x)
x = resnet_model(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)  # dropout for regularization
outputs = layers.Dense(1, activation='sigmoid')(x)

# Create the model
model = models.Model(inputs, outputs)

base_learning_rate = 0.001

# since this is binary classification, loss is BCE
model.compile(optimizer=Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])

# Train with frozen base model layers
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)

In [ ]:
# what are the top layers?
resnet_model_top = ResNet50(weights='imagenet', include_top=True, input_shape=(224, 224, 3))
for layer in resnet_model_top.layers[-2:]:
  print(layer)

In [ ]:
# RESNET50 - Training 50 epochs and unfreeze last 10 layers

# delete top layer (include_top = false, make other layers non-trainable)
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in resnet_model.layers[:-10]:
  layer.trainable = False

inputs = tf.keras.Input(shape=(224, 224, 3))
x = data_augmentation_layer(inputs)
x = normalization_layer(x)
x = resnet_model(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model = models.Model(inputs, outputs)

base_learning_rate = 0.001

model.compile(optimizer=Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50
)

# evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_ds)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# get the true labels for the test dataset
y_true = np.concatenate([labels.numpy() for _, labels in test_ds])
y_pred = model.predict(test_ds)  # Predicted probabilities
y_pred_labels = (y_pred > 0.5).astype(int)
y_pred_classes = np.argmax(y_pred_labels, axis=1)  # Convert to class labels

conf_matrix = confusion_matrix(y_true, y_pred_labels)

print("Confusion Matrix:")
print(conf_matrix)

plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix for Binary Classification")
plt.show()

# AlexNet

In [ ]:
# ALEXNET Data Pre-Processing Baseline Model

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader

# Define transformations: resizing, normalization ...
transform = transforms.Compose([
    transforms.Resize(256),       # Resize the image to 256x256
    transforms.CenterCrop(224),   # Crop the center of the image to 224x224
    transforms.ToTensor(),        # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization for AlexNet
])

# train data
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# valid data
valid_dataset = datasets.ImageFolder(root=val_dir, transform=transform)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=True)

In [ ]:
# ALEXNET Baseline Model

# load pre-trained AlexNet
alexnet = models.alexnet(pretrained=True)

# freeze early layers
for param in alexnet.features.parameters():
    param.requires_grad = False

# modify the classifier for binary classification
alexnet.classifier[6] = nn.Linear(alexnet.classifier[6].in_features, 2)

# move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
alexnet = alexnet.to(device)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(alexnet.parameters(), lr=0.001)

# training loop
num_epochs = 5
for epoch in range(num_epochs):
    alexnet.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # forward pass
        outputs = alexnet(images)
        loss = criterion(outputs, labels)

        # backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # print statistics for this epoch
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

    # validation phase
    alexnet.eval()  # set the model to evaluation mode
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():  # disable gradient computation for validation
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)

            # forward pass
            outputs = alexnet(images)
            loss = criterion(outputs, labels)

            # update validation metrics
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_loss /= len(valid_loader)
    val_accuracy = 100 * correct_val / total_val

    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

In [ ]:
# ALEXNET Modified Baseline

# load pre-trained AlexNet
alexnet = models.alexnet(pretrained=True)

# freeze early layers
for param in alexnet.features.parameters():
    param.requires_grad = False

# modify the classifier
alexnet.classifier = nn.Sequential(
    *list(alexnet.classifier.children())[:-1],  # Remove the last layer
    nn.Linear(alexnet.classifier[6].in_features, 128),  # Add dense layer
    nn.ReLU(),
    nn.Linear(128, 2)  # Final output layer for 2 classes
)

# move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
alexnet = alexnet.to(device)

# loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(alexnet.parameters(), lr=0.001)

# training loop
num_epochs = 5
for epoch in range(num_epochs):
    alexnet.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # forward pass
        outputs = alexnet(images)
        loss = criterion(outputs, labels)

        # backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # print statistics for this epoch
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

    # validation phase
    alexnet.eval()  # set the model to evaluation mode
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():  # disable gradient computation for validation
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)

            # forward pass
            outputs = alexnet(images)
            loss = criterion(outputs, labels)

            # update validation metrics
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_loss /= len(valid_loader)
    val_accuracy = 100 * correct_val / total_val

    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

In [ ]:
# ALEXNET Experiment A

# load pre-trained AlexNet
alexnet = models.alexnet(pretrained=True)

# freeze early layers (optional)
for param in alexnet.features.parameters():
    param.requires_grad = False

# modify the classifier
alexnet.classifier = nn.Sequential(
    *list(alexnet.classifier.children())[:-1],  # remove the last layer
    nn.Linear(alexnet.classifier[6].in_features, 128),  # add dense layer
    nn.ReLU(),  # ReLU activation
    nn.Linear(128, 128),  # Add dense layer
    nn.ReLU(),  # ReLU activation
    nn.Linear(128, 2)  # Final output layer for 2 classes
)

# move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
alexnet = alexnet.to(device)

# loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(alexnet.parameters(), lr=0.001)

num_epochs = 5
for epoch in range(num_epochs):
    alexnet.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = alexnet(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

    # validation phase
    alexnet.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = alexnet(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_loss /= len(valid_loader)
    val_accuracy = 100 * correct_val / total_val

    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

In [ ]:
# ALEXNET Experiment B

# Load pre-trained AlexNet
alexnet = models.alexnet(pretrained=True)

# Freeze early layers (optional)
for param in alexnet.features.parameters():
    param.requires_grad = False

# Modify the classifier
alexnet.classifier = nn.Sequential(
    *list(alexnet.classifier.children())[:-1],  # Remove the last layer
    nn.Linear(alexnet.classifier[6].in_features, 256),  # Add dense layer
    nn.ReLU(),  # ReLU activation
    nn.Linear(256, 2)  # Final output layer for 2 classes
)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
alexnet = alexnet.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(alexnet.parameters(), lr=0.001)

# Training loop (simplified)
num_epochs = 5
for epoch in range(num_epochs):
    alexnet.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = alexnet(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Print statistics for this epoch
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

    # Validation phase
    alexnet.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():  # Disable gradient computation for validation
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = alexnet(images)
            loss = criterion(outputs, labels)

            # Update validation metrics
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_loss /= len(valid_loader)
    val_accuracy = 100 * correct_val / total_val

    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

In [ ]:
# ALEXNET Experiment C

# Load pre-trained AlexNet
alexnet = models.alexnet(pretrained=True)

for param in alexnet.features.parameters():
    param.requires_grad = False

# Modify the classifier
alexnet.classifier = nn.Sequential(
    *list(alexnet.classifier.children())[:-1],
    nn.Linear(alexnet.classifier[6].in_features, 128),  # dense layer with 128 unit outputs
    nn.ReLU(), # relu activation
    nn.Dropout(p=0.5),  # dropout with a probability of 0.5
    nn.Linear(128, 2)  # final output layer for 2 classes
)

# move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
alexnet = alexnet.to(device)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(alexnet.parameters(), lr=0.001)

num_epochs = 5
for epoch in range(num_epochs):
    alexnet.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # forward pass
        outputs = alexnet(images)
        loss = criterion(outputs, labels)

        # backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

    # validation phase
    alexnet.eval()  # set the model to evaluation mode
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():  # disable gradient computation for validation
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)

            # forward pass
            outputs = alexnet(images)
            loss = criterion(outputs, labels)

            # update validation metrics
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_loss /= len(valid_loader)
    val_accuracy = 100 * correct_val / total_val

    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

# InceptionNet

In [ ]:
# INCEPTIONNET - Baseline Model

# Load InceptionNet without the top layer
inceptionnet_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

for layer in inceptionnet_model.layers:
  layer.trainable = False

model = models.Sequential([
    layers.Input(shape=(299, 299, 3)),
    data_augmentation_layer,
    layers.Lambda(lambda x: tf.image.resize(x, (299, 299))),
    layers.Lambda(lambda x: tf.keras.applications.inception_v3.preprocess_input(x)), # normalization
    inceptionnet_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])

base_learning_rate = 0.001

# since this is binary classification, loss is BCE
model.compile(optimizer=Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)

In [ ]:
# INCEPTIONNET Experiment with drop out 1

inceptionnet_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

for layer in inceptionnet_model.layers:
  layer.trainable = False

model = models.Sequential([
    layers.Input(shape=(299, 299, 3)),

    data_augmentation_layer,
    layers.Lambda(lambda x: tf.image.resize(x, (299, 299))),
    layers.Lambda(lambda x: tf.keras.applications.inception_v3.preprocess_input(x)), # normalization

    inceptionnet_model,
    layers.Dropout(0.5),
    layers.GlobalAveragePooling2D(),
    layers.Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])

base_learning_rate = 0.001
model.compile(optimizer=Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)

In [ ]:
# INCEPTIONNET - Experiment with drop out 2

# Load InceptionNet without the top layer
inceptionnet_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

for layer in inceptionnet_model.layers:
  layer.trainable = False

model = models.Sequential([
    layers.Input(shape=(299, 299, 3)),  # Input size
    data_augmentation_layer,
    layers.Lambda(lambda x: tf.image.resize(x, (299, 299))),
    layers.Lambda(lambda x: tf.keras.applications.inception_v3.preprocess_input(x)), # normalization
    inceptionnet_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),  # Dropout layer to reduce overfitting
    layers.Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])

base_learning_rate = 0.001

# since this is binary classification, loss is BCE
model.compile(optimizer=Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)

In [ ]:
# INCEPTIONNET - Experiment with more data augmentation layers

data_augmentation_more_layers = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomContrast(0.2)
])

inceptionnet_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

for layer in inceptionnet_model.layers:
  layer.trainable = False

model = models.Sequential([
    layers.Input(shape=(299, 299, 3)),
    data_augmentation_more_layers,
    layers.Lambda(lambda x: tf.image.resize(x, (299, 299))),
    layers.Lambda(lambda x: tf.keras.applications.inception_v3.preprocess_input(x)), # normalization

    inceptionnet_model,
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.GlobalAveragePooling2D(),
    layers.Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])

base_learning_rate = 0.001

# since this is binary classification, loss is BCE
model.compile(optimizer=Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)